## Centralized Variable Star Catalog

For our task to create a centralized data table for classified variable stars, we will be using astroquery, simbad/visier to retrieve data tables from different papers / sources in a pythonic format

### 1. Necessary imports
   

In [1]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as units
import csv
import pandas as pd

### 2. Query for the catalogs of interest

In [2]:
# set the row limit for how many rows to retrieve (default is 50)
Vizier.ROW_LIMIT = -1

# load the catalog for our tables of interest

# we will first start with the RRLyrae, Cepheids, and candidate LPVs from Bersier and Wood's paper
catalog_list = Vizier.get_catalogs('J/AJ/123/840')

In [3]:
# print table names
print(catalog_list.keys())

# access a specific table
table = catalog_list[0]

['J/AJ/123/840/table1', 'J/AJ/123/840/table2', 'J/AJ/123/840/table3', 'J/AJ/123/840/table4']


In [4]:
print(table) # this table corresponds to the journal of observations for each field

Field    RJDV     RJDIc   Obs
          d         d        
----- --------- --------- ---
    1 652.26667 652.26667 MSO
    1 657.26976 657.26976 MSO
    1 658.27549 658.27549 MSO
    1 659.28311 659.28311 MSO
    1 660.25075 660.25075 MSO
    1 661.23904 661.23904 MSO
    1 662.23573 662.23573 MSO
    1 663.23345 663.23345 MSO
    1 664.23512 664.23512 MSO
  ...       ...       ... ...
    4 747.56197 747.57163 SSO
    4 747.68075 747.67128 SSO
    4 749.43694 749.42517 SSO
    4 749.54685 749.55630 SSO
    4 785.49755 785.51659 SSO
    4 785.59975 785.61305 SSO
    4 786.54580 786.55536 SSO
    4 786.62969 786.63918 SSO
    4 787.45850 787.47754 SSO
    4 788.63089 788.64039 SSO
Length = 132 rows


In [5]:
# we specifically want the tables for the variable stars
RRLyrae_Bersier_Wood = catalog_list[1]
AC_P2C_Bersier_Wood = catalog_list[2]
LPV_Cand_Bersier_Wood = catalog_list[3]

In [55]:
RRLyrae_Bersier_Wood.info()
AC_P2C_Bersier_Wood.info()
LPV_Cand_Bersier_Wood.info()

<Table length=515>
  name   dtype  unit  format                        description                       
------- ------- ---- ------- ---------------------------------------------------------
    FBW   str16                                   RR Lyrae name (FBW JHHMMSS.s+DDMMSS)
 o_Vmag   int16                                    Number of observations used in Vmag
   Vmag float32  mag {:6.3f}                       The phase-weighted V band magnitude
 e_Vmag float32  mag {:6.3f}                                   The uncertainty in Vmag
o_Icmag   int16                                   Number of observations used in Icmag
  Icmag float32  mag {:6.3f}                      The phase-weighted Ic band magnitude
e_Icmag float32  mag {:6.3f}                                  The uncertainty in Icmag
     WS float32      {:6.3f} The Welch-Stetson (1993AJ....105.1813W) variability index
 Period float64    d {:8.5f}                                          Pulsation period
 E(B-V) float32  mag {:6

In [96]:
# we can easily convert the table to csv to store on our computer using a few python lines
csv_filename = "RRLyrae_Bersier_Wood.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(RRLyrae_Bersier_Wood.colnames) #header
    for row in RRLyrae_Bersier_Wood: #rows
        writer.writerow([row[col] for col in RRLyrae_Bersier_Wood.colnames])

print(f"Catalog written to {csv_filename}")

Catalog written to RRLyrae_Bersier_Wood.csv


In [97]:
# there is also a astropy method to do this which is shorter
AC_P2C_Bersier_Wood.write('Cepheid_Bersier_Wood.csv', format='csv', overwrite=True)
LPV_Cand_Bersier_Wood.write('VStar_CatalogData/LPV_Cand_Bersier_Wood.csv', format='csv', overwrite=True)

In [7]:
# load some LPVs from the Gaia LPV catalog in a random region (just a test)
center = SkyCoord(ra=100.4025, dec=-50.9661, unit='deg')
fornax_center = SkyCoord(ra='39.9992', dec='34.4492', unit=(units.degree, units.degree), frame='icrs')
radius = 1000 * units.arcmin
fornax_radius = 1 * units.deg



query = Vizier.query_region(center, radius=radius, catalog='J/ApJS/264/20')
query_fornax = Vizier.query_region(fornax_center, radius=radius, catalog='J/ApJS/264/20')

In [9]:
print(query.keys())
print(query_fornax.keys())

['J/ApJS/264/20/table1']
[]


In [100]:
query.pprint()

TableList with 1 tables:
	'0:J/ApJS/264/20/table1' with 34 column(s) and 17 row(s) 


In [10]:
result = query[0]

In [11]:
result.info()

<Table length=17>
  name    dtype  unit  format                                 description                                 n_bad
-------- ------- ---- -------- -------------------------------------------------------------------------- -----
    OGLE   str14                                        Star identifier as in Iwanek+ 2022, J/ApJS/260/46     0
 RAJ2000   str11                                                   [5/19] Hour of Right ascension (J2000)     0
 DEJ2000   str11                                                            Degree of Declination (J2000)     0
    Type    str1                                                 Surface chemistry type of Miras (O or C)     0
    Imag float32  mag  {:6.3f}                                      [9.5/20.9] Mean OGLE I-band magnitude     0
    Iamp float32  mag  {:6.3f}                                  [0.8/23] OGLE I-band brightness amplitude     0
    Vmag float32  mag  {:7.3f}                                    [12.7/23.3]? Mean OG

In [12]:
print(result)

     OGLE       RAJ2000     DEJ2000   Type ...    Dist       mu     Mask  Simbad
                                           ...     pc       mag                 
------------- ----------- ----------- ---- ... ---------- ------- ------- ------
GD-LPV-000267 07 49 40.85 -39 33 29.1    C ...   2144.000  11.656 -99.999 Simbad
GD-LPV-000276 07 52 48.55 -40 47 47.5    O ...   6286.000  13.992 -99.999 Simbad
GD-LPV-000283 07 56 26.39 -41 13 58.4    O ...   6188.000  13.958 -99.999 Simbad
GD-LPV-000287 07 57 24.69 -42 30 36.1    O ...   5000.000  13.495 -99.999 Simbad
GD-LPV-000302 08 01 50.04 -41 49 45.1    O ...   2620.000  12.092 -99.999 Simbad
GD-LPV-000310 08 03 10.71 -42 00 35.4    O ...   9117.000  14.799 -99.999 Simbad
GD-LPV-000313 08 05 03.56 -44 15 58.7    O ...   6402.000  14.032 -99.999 Simbad
GD-LPV-000322 08 07 14.38 -42 56 34.3    O ...   3646.000  12.809 -99.999 Simbad
GD-LPV-000326 08 08 45.46 -44 33 06.4    O ...   6660.000  14.117 -99.999 Simbad
GD-LPV-000327 08 09 05.18 -4

In [104]:
result.write('MirasOGLE.csv', format='csv', overwrite=True)

In [13]:
# query from the gaia catalog
from astroquery.gaia import Gaia

In [14]:
# test, query a square region in the sky for sources
coord = SkyCoord(ra=280, dec=-60, unit=(units.degree, units.degree), frame='icrs')
width = units.Quantity(0.1, units.deg)
height = units.Quantity(0.1, units.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

r.pprint(max_lines=12, max_width=130)

INFO: Query finished. [astroquery.utils.tap.core]


         dist             solution_id             designation          ... ebpminrp_gspphot_upper libname_gspphot
                                                                       ...          mag                          
--------------------- ------------------- ---------------------------- ... ---------------------- ---------------
0.0026043272506261527 1636148068921376768 Gaia DR3 6636090334814214528 ...                     --                
0.0033616678530916998 1636148068921376768 Gaia DR3 6636090339112400000 ...                     --                
0.0038498801828703495 1636148068921376768 Gaia DR3 6636090339113063296 ...                     --                
                  ...                 ...                          ... ...                    ...             ...
 0.019751317240143573 1636148068921376768 Gaia DR3 6636090407832546944 ...                 0.1176           MARCS
 0.019916769172899054 1636148068921376768 Gaia DR3 6636066940132132352 ...              

In [15]:
Gaia.ROW_LIMIT = 50
j = Gaia.cone_search_async(fornax_center, radius=units.Quantity(1.0, units.deg)) # do a cone search for sources in fornax

INFO: Query finished. [astroquery.utils.tap.core]


In [16]:
results = j.get_results()

In [17]:
columns = list(results.columns)
# there is 153 columns, not going to print it out
print(len(columns))
print(columns[:10])

153
['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax']


In [18]:
results['classprob_dsc_combmod_star']

0.9999935
0.99995106
0.9999985
0.99996364
0.9999271
0.99994254
0.979556
0.9999956
0.99999636
0.99886584
0.9999747


In [19]:
# load gaia_source and vari_summary tables
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')
gaiadr3_var = Gaia.load_table('gaiadr3.vari_summary')

In [20]:
print(str(gaiadr3_table) + '\n')
print(gaiadr3_var)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152

TAP Table name: gaiadr3.vari_summary
Description: Summary table that provides the information on where a {\tt sourceId} can be found in the different {\tt Variability} tables and statistical parameters of time series, using only transits not rejected.

Note that NULL is reported when the statistical parame

In [21]:
# test a adql query sychronously
query = f'''SELECT TOP 100 solution_id,ref_epoch,ra_dec_corr,astrometric_n_obs_al,
            matched_transits,duplicated_source,phot_variable_flag
            from gaiadr3.gaia_source order by source_id'''
job = Gaia.launch_job(query)

In [22]:
r = job.get_results()
print(r['ra_dec_corr']) # works correctly

ra_dec_corr 
------------
  0.12293493
  0.16325329
   0.1152631
  0.03106277
 0.090631574
  0.25799984
  0.15041357
  0.15176746
  0.19033876
  0.18675442
         ...
-0.047490653
  0.18519369
  0.11701631
  0.14461127
  0.05615686
  0.26646927
-0.019807748
  0.81679803
 -0.07291612
 -0.12864673
Length = 100 rows


In [23]:
print(job) # can inspect the job

<Table length=100>
        name          dtype  unit                          description                         
-------------------- ------- ---- -------------------------------------------------------------
         solution_id   int64                                                Solution Identifier
           ref_epoch float64   yr                                               Reference epoch
         ra_dec_corr float32                Correlation between right ascension and declination
astrometric_n_obs_al   int16      Total number of observations in the along-scan (AL) direction
    matched_transits   int16                      The number of transits matched to this source
   duplicated_source    bool                            Source with multiple source identifiers
  phot_variable_flag  object                                       Photometric variability flag
Jobid: None
Phase: COMPLETED
Owner: None
Output file: 1750961256901O-result.vot.gz
Results: None


In [24]:
# count the # of rr lyrae in the variability table
query = "SELECT COUNT(*) FROM gaiadr3.vari_rrlyrae"
job = Gaia.launch_job_async(query)
result = job.get_results()
print(result['COUNT_ALL'])

INFO: Query finished. [astroquery.utils.tap.core]


COUNT_ALL
---------
   271779


In [27]:
# create asynchronous queries to get data from variability tables for rrl, ceph, LPV
# query_rrl = f'''SELECT TOP 1000 r.source_id, r.pf, r.int_average_g, r.int_average_bp, r.int_average_rp,
#             g.ra, g.dec, g.parallax, g.bp_rp
#             FROM gaiadr3.vari_rrlyrae AS r
#             JOIN gaiadr3.gaia_source AS g
#             ON r.source_id = g.source_id
#             WHERE g.ra BETWEEN 39.3 AND 40.7
#             AND g.dec BETWEEN -34.9 AND -33.8
#             '''
radius = 1 * units.deg

query_rrl = f'''SELECT TOP 1000 r.source_id, r.pf, r.int_average_g, r.int_average_bp, r.int_average_rp,
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_rrlyrae AS r
            JOIN gaiadr3.gaia_source AS g
            ON r.source_id = g.source_id
            WHERE CONTAINS(
                POINT('ICRS', ra, dec),
                CIRCLE('ICRS', {fornax_center.ra.deg}, {fornax_center.dec.deg}, {radius.to(units.deg).value})
            ) = 1
            '''

job = Gaia.launch_job_async(query_rrl)
job.get_results().write('rrl_test.csv', format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


In [28]:
print(job)

<Table length=2>
     name       dtype  unit                                   description                                    n_bad
-------------- ------- ---- -------------------------------------------------------------------------------- -----
     source_id   int64                                                              Unique source identifier     0
            pf float64    d Period corresponding to the fundamental pulsation mode in the G band time series     2
 int_average_g float32  mag                                       Intensity-averaged magnitude in the G band     0
int_average_bp float32  mag                                      Intensity-averaged magnitude in the BP band     0
int_average_rp float32  mag                                      Intensity-averaged magnitude in the RP band     0
            ra float64  deg                                                                  Right ascension     0
           dec float64  deg                                    

### Retrieve sample of RR Lyrae from Gaia with known periods

In [29]:
# query for a sample of rr lyrae with known periods
query_rrl_sample = f'''SELECT TOP 100 r.source_id, r.pf, r.int_average_g, r.int_average_bp, r.int_average_rp,
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_rrlyrae AS r
            JOIN gaiadr3.gaia_source AS g
            ON r.source_id = g.source_id
            WHERE r.pf IS NOT NULL
            '''

In [30]:
job = Gaia.launch_job_async(query_rrl_sample)
r = job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]


In [32]:
print(job)
r.write('gaia_rrl_sample.csv', format='csv', overwrite=True) # write to a csv

<Table length=100>
     name       dtype  unit                                   description                                    n_bad
-------------- ------- ---- -------------------------------------------------------------------------------- -----
     source_id   int64                                                              Unique source identifier     0
            pf float64    d Period corresponding to the fundamental pulsation mode in the G band time series     0
 int_average_g float32  mag                                       Intensity-averaged magnitude in the G band     0
int_average_bp float32  mag                                      Intensity-averaged magnitude in the BP band    26
int_average_rp float32  mag                                      Intensity-averaged magnitude in the RP band    26
            ra float64  deg                                                                  Right ascension     0
           dec float64  deg                                  

In [33]:
# count the # of rr lyrae in the table
query = "SELECT COUNT(*) FROM gaiadr3.vari_cepheid"
job = Gaia.launch_job_async(query)
result = job.get_results()
print(result['COUNT_ALL'])

INFO: Query finished. [astroquery.utils.tap.core]


COUNT_ALL
---------
    15021


In [43]:
# create asynchronous queries to get data from variability tables for rrl, ceph, LPV
# query_rrl = f'''SELECT TOP 1000 r.source_id, r.pf, r.int_average_g, r.int_average_bp, r.int_average_rp,
#             g.ra, g.dec, g.parallax, g.bp_rp
#             FROM gaiadr3.vari_cepheid AS r
#             JOIN gaiadr3.gaia_source AS g
#             ON r.source_id = g.source_id
#             WHERE g.ra BETWEEN 39.3 AND 40.7
#             AND g.dec BETWEEN -34.9 AND -33.8
#             '''

radius = 1 * units.deg

query_cepheid = f'''SELECT TOP 1000 c.source_id, c.pf, c.int_average_g, c.int_average_bp, c.int_average_rp,
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_cepheid AS c
            JOIN gaiadr3.gaia_source AS g
            ON c.source_id = g.source_id
            WHERE CONTAINS(
                POINT('ICRS', ra, dec),
                CIRCLE('ICRS', {fornax_center.ra.deg}, {fornax_center.dec.deg}, {radius.to(units.deg).value})
            ) = 1
            '''

job = Gaia.launch_job_async(query_cepheid)
job.get_results().write('cepheid_test.csv', format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


In [44]:
print(job)

<Table length=0>
     name       dtype  unit                                   description                                   
-------------- ------- ---- --------------------------------------------------------------------------------
     source_id   int64                                                              Unique source identifier
            pf float64    d Period corresponding to the fundamental pulsation mode in the G band time series
 int_average_g float32  mag                                       Intensity-averaged magnitude in the G band
int_average_bp float32  mag                                      Intensity-averaged magnitude in the BP band
int_average_rp float32  mag                                      Intensity-averaged magnitude in the RP band
            ra float64  deg                                                                  Right ascension
           dec float64  deg                                                                      Declination
  

### Retrieve sample of Cepheids from Gaia with known periods

In [48]:
query_ceph_sample = f'''SELECT TOP 100 c.source_id, c.pf, c.int_average_g, c.int_average_bp, c.int_average_rp,
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_cepheid AS c
            JOIN gaiadr3.gaia_source AS g
            ON c.source_id = g.source_id
            WHERE c.pf IS NOT NULL
            '''

In [49]:
job = Gaia.launch_job_async(query_ceph_sample)
job.get_results().write('gaia_ceph_sample.csv', format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


In [46]:
query = "SELECT COUNT(*) FROM gaiadr3.vari_long_period_variable"
job = Gaia.launch_job_async(query)
result = job.get_results()
print(result['COUNT_ALL'])

INFO: Query finished. [astroquery.utils.tap.core]


COUNT_ALL
---------
  1720588


In [52]:
# create asynchronous queries to get data from variability tables for rrl, ceph, LPV
# query_rrl = f'''SELECT TOP 1000 r.source_id, r.pf, r.int_average_g, r.int_average_bp, r.int_average_rp,
#             g.ra, g.dec, g.parallax, g.bp_rp
#             FROM gaiadr3.vari_long_period_variable AS r
#             JOIN gaiadr3.gaia_source AS g
#             ON r.source_id = g.source_id
#             WHERE g.ra BETWEEN 39.3 AND 40.7
#             AND g.dec BETWEEN -34.9 AND -33.8
#             '''

radius = 1 * units.deg

query_lpv = f'''SELECT TOP 1000 l.source_id, l.frequency, l.amplitude, 
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_long_period_variable AS l
            JOIN gaiadr3.gaia_source AS g
            ON l.source_id = g.source_id
            WHERE CONTAINS(
                POINT('ICRS', ra, dec),
                CIRCLE('ICRS', {fornax_center.ra.deg}, {fornax_center.dec.deg}, {radius.to(units.deg).value})
            ) = 1
            '''

job = Gaia.launch_job_async(query_lpv)
job.get_results().write('lpv_test.csv', format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


In [53]:
print(job)

<Table length=5>
   name    dtype   unit           description            n_bad
--------- ------- ----- -------------------------------- -----
source_id   int64               Unique source identifier     0
frequency float64 1 / d             Frequency of the LPV     3
amplitude float32   mag Amplitude of the LPV variability     3
       ra float64   deg                  Right ascension     0
      dec float64   deg                      Declination     0
 parallax float64   mas                         Parallax     0
    bp_rp float32   mag                   BP - RP colour     0
Jobid: 1750962459506O
Phase: COMPLETED
Owner: None
Output file: async_20250626182739.vot
Results: None


### Same for LPVs with known frequency (columns more limited here):

In [54]:
query_lpv_sample = f'''SELECT TOP 100 l.source_id, l.frequency, l.amplitude, 
            g.ra, g.dec, g.parallax, g.bp_rp
            FROM gaiadr3.vari_long_period_variable AS l
            JOIN gaiadr3.gaia_source AS g
            ON l.source_id = g.source_id
            WHERE l.frequency IS NOT NULL
            '''

job = Gaia.launch_job_async(query_lpv_sample)
job.get_results().write('gaia_lpv_sample.csv', format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]
